In [ ]:
#clear all
%reset -f

#import packages
import numpy as np
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'frn'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [ ]:
#setting up path/ directory
#access specific file for specific participant
def load_epochs(pp_num, root_dir, erp_path):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'mra_p%03d_run0-epo.fif' % pp_num)

    epochs = mne.read_epochs(filename)
    #baseline for each trial
    if erp_path == 'frn':
        epochs = epochs.apply_baseline(baseline = (-1.5, -1.2))
    elif erp_path == 'lrp':
        epochs = epochs.apply_baseline(baseline = (-1.3, -1.0))
    elif erp_path == 'ern':
        epochs = epochs.apply_baseline(baseline = (-1.35, -1.05))

    return epochs, pp_directory

In [ ]:
#separate function to get indexed epochs for random, since we have them before rotation or mirror
def trial_list_rdm(task, pp, index):
    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rdmrot':
            #get trial after first task (trial 48 to 95, 234 to 281)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
                    
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
    elif pp % 2 == 1:   
        if task == 'rdmrot':
            #get trial after first five tasks (trial 234 to 281)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

    
    return(triallist)
    

In [ ]:
#get matching trials from eeg data, where we account for missing triggers
def epoch_trials(task, pp, epochs, index):

    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    if task == 'aln':
        #get trial numbers where tasktrig is 16149 and trigidx is 1
        tasksdf = df[0:48]
        subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16149]
        triallist = []
        for i in index:
            ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
            nint = int(ndat['trigidx'])
            if nint == 1:
                trial = int(ndat['ctrial'])
                triallist.append(trial)

        epochs = epochs[triallist]
    
    #for the other tasks, the order depends on their pp number
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rot':
            #rot trials are after first two tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'mir':
            #get trial after first task (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
    elif pp % 2 == 1:
        if task == 'rot':
            #get trial after first five tasks (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'mir':
            #get trial after first five tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
    
    return(epochs)

In [ ]:
#save epochs
def output_condition_epochs(pp_num, data, ppdir, condname, task, training):
    # Save the cleaned data :)
    out_fname = os.path.join(ppdir, 'p%03d_EarlyLate_%s_%s_%s-epo.fif' % (pp_num, condname, task, training))
    data.save(out_fname, overwrite = True)

In [ ]:
#save as an evoked object
def output_condition_evoked(pp_num, data, ppdir, condname, task, training):
    evoked = data.average()
    out_fname = os.path.join(ppdir, 'p%03d_EarlyLate_%s_%s_%s-ave.fif' % (pp_num, condname, task, training))
    evoked.save(out_fname, overwrite = True)

In [ ]:
#save as evoked objects
#each perturbation condition can have separate loops
l_cutoff = 30

b0 = list(range(0,48))

# ALIGNED
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    aldata = epoch_trials('aln', pp, data, b0)
    #print(pp, numtrials)
    output_condition_epochs(pp, aldata, ppdir, 'P3', 'aligned', 'all')
    #filter at 30 Hz
    aldata = aldata.filter(l_freq= None, h_freq=l_cutoff)
    #then average across trials
    output_condition_evoked(pp, aldata, ppdir, 'P3', 'aligned', 'all')

In [ ]:
#get grand averages - ALIGNED
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
#aligned

flist = []
evoked_list = []
for pp in participants:
    id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
    pp_directory = os.path.join(id_directory, erps)
    fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_%s-ave.fif' % (participants[pp], 'P3', 'aligned', 'all'))
    evoked = mne.read_evokeds(fname)
    flist.append(evoked)
    evoked = flist[pp][0]
    evoked_list.append(evoked)
aln_b0_grand = mne.grand_average(evoked_list)
aln_b0_flist = evoked_list

In [ ]:
#save as evoked objects
l_cutoff = 30

#ROTATED and MIRROR TRAINING
tasks = ['rot', 'mir']

#split into training blocks
#18 trials per block -> 5 blocks
b0 = list(range(0,12))
b1 = list(range(54,90))

training = [b0, b1]

       
#rot and mir
for task in range(0, len(tasks)):
    for pp in participants:
        data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
        for t in range(0, len(training)):
            pdata = epoch_trials(tasks[task], pp, data, training[t])
            output_condition_epochs(pp, pdata, ppdir, 'P3', tasks[task], t)
            #filter at 30 Hz
            pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
            #then average across trials
            output_condition_evoked(pp, pdata, ppdir, 'P3', tasks[task], t)

In [ ]:
#get grand averages - ROTATED AND MIRROR
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
tasks = ['rot', 'mir']
training = [0, 1]
#rot and mir
for task in range(0, len(tasks)):
    for t in range(0, len(training)):
        flist = []
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_%s-ave.fif' % (participants[pp], 'P3', tasks[task], training[t]))
            if os.path.exists(fname) == False:
                continue
            evoked = mne.read_evokeds(fname)
            flist.append(evoked)
            evoked = evoked[0]
            evoked_list.append(evoked)
        if task == 0 and t == 0:
            rot_b0_grand = mne.grand_average(evoked_list)
            rot_b0_flist = evoked_list
        elif task == 0 and t == 1:
            rot_b1_grand = mne.grand_average(evoked_list)
            rot_b1_flist = evoked_list
        elif task == 1 and t == 0:
            mir_b0_grand = mne.grand_average(evoked_list)
            mir_b0_flist = evoked_list
        elif task == 1 and t == 1:
            mir_b1_grand = mne.grand_average(evoked_list)
            mir_b1_flist = evoked_list

In [ ]:
#save as evoked objects
l_cutoff = 30

#RANDOM 
#combine rdm rot and rdm mir into one per participant, but get correct indices
# early is now defined by the first rdm block they do, late will be the last rdm block they do

#split into training blocks
b0 = list(range(0,12))
b1 = list(range(24,48))
training = [b0, b1]

for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    for t in range(0, len(training)):
        if pp % 2 == 0 and t == 0:
            rdm_early_trials = trial_list_rdm('rdmrot', pp, training[0])
            pdata = data[rdm_early_trials]
        elif pp % 2 == 0 and t == 1:
            rdm_late_trials = trial_list_rdm('rdmmir', pp, training[1])
            pdata = data[rdm_late_trials]
        elif pp % 2 == 1 and t == 0:
            rdm_early_trials = trial_list_rdm('rdmmir', pp, training[0])
            pdata = data[rdm_early_trials]
        elif pp % 2 == 1 and t == 1:
            rdm_late_trials = trial_list_rdm('rdmrot', pp, training[1])
            pdata = data[rdm_late_trials]
        output_condition_epochs(pp, pdata, ppdir, 'P3', 'rdm', t)
        #filter at 30 Hz
        pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(pp, pdata, ppdir, 'P3', 'rdm', t)

# Deprecated----
# for pp in participants:
#     data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
#     #rdmrot
#     rdmrot_triallist = trial_list_rdm('rdmrot', pp, training[0])
#     print(rdmrot_triallist)
#     #rdmmir
#     rdmmir_triallist = trial_list_rdm('rdmmir', pp, training[0])
#     print(rdmmir_triallist)
    
#     if pp % 2 == 0:
#         # Save early
#         rdm_early_trials = rdmrot_triallist
#         pdata = data[rdm_early_trials]
#         output_condition_epochs(pp, pdata, ppdir, 'P3', 'rdm', '0')
#         #filter at 30 Hz
#         pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
#         #then average across trials
#         output_condition_evoked(pp, pdata, ppdir, 'P3', 'rdm', '0')
#         # Save late
#         rdm_late_trials = rdmmir_triallist
#         pdata = data[rdm_late_trials]
#         output_condition_epochs(pp, pdata, ppdir, 'P3', 'rdm', '1')
#         #filter at 30 Hz
#         pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
#         #then average across trials
#         output_condition_evoked(pp, pdata, ppdir, 'P3', 'rdm', '1')
#     elif pp % 2 == 1:
#         # Save early
#         rdm_early_trials = rdmmir_triallist
#         pdata = data[rdm_early_trials]
#         output_condition_epochs(pp, pdata, ppdir, 'P3', 'rdm', '0')
#         #filter at 30 Hz
#         pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
#         #then average across trials
#         output_condition_evoked(pp, pdata, ppdir, 'P3', 'rdm', '0')
#         # Save late
#         rdm_late_trials = rdmrot_triallist
#         pdata = data[rdm_late_trials]
#         output_condition_epochs(pp, pdata, ppdir, 'P3', 'rdm', '1')
#         #filter at 30 Hz
#         pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
#         #then average across trials
#         output_condition_evoked(pp, pdata, ppdir, 'P3', 'rdm', '1')


# # Deprecated ----
# #RANDOM

# #split into training blocks
# b0 = list(range(0,24))
# b1 = list(range(24,48))

# training = [b0, b1]
            
# # RANDOM (could be optimized, but in case we want to separate rdm before rot or mir, we have it as below)
# for pp in participants:
#     data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
#     for t in range(0, len(training)):
#         #rdmrot
#         rdmrot_triallist = trial_list_rdm('rdmrot', pp, training[t])
#         #rdmmir
#         rdmmir_triallist = trial_list_rdm('rdmmir', pp, training[t])
    
#         #combine both rdmrot and rdmmir accdg to conditions
#         triallist_rdm = rdmrot_triallist + rdmmir_triallist
#         triallist_rdm.sort()
#         #save as one file of evokeds called rdm
#         pdata = data[triallist_rdm]

#         output_condition_epochs(pp, pdata, ppdir, 'P3', 'rdm', t)
#         #filter at 30 Hz
#         pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
#         #then average across trials
#         output_condition_evoked(pp, pdata, ppdir, 'P3', 'rdm', t)

In [ ]:
#get grand averages - RANDOM
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
tasks = ['rdm']
training = [0, 1]

#rdm
for task in range(0, len(tasks)):
    for t in range(0, len(training)):
        flist = []
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_%s-ave.fif' % (participants[pp], 'P3', tasks[task], training[t]))
            if os.path.exists(fname) == False:
                continue
            evoked = mne.read_evokeds(fname)
            flist.append(evoked)
            evoked = evoked[0]
            evoked_list.append(evoked)
        if task == 0 and t == 0:
            rdm_b0_grand = mne.grand_average(evoked_list)
            rdm_b0_flist = evoked_list
        elif task == 0 and t == 1:
            rdm_b1_grand = mne.grand_average(evoked_list)
            rdm_b1_flist = evoked_list

In [ ]:
#convert evokeds to data frame, so we can plot the ERPs better

flists = ['aln', 
          'rot_b0', 'rot_b1',
          'mir_b0', 'mir_b1',
          'rdm_b0', 'rdm_b1']

for f in flists:
    newdf = pd.DataFrame()
    for pp in participants:
        if f == 'aln':
            df = aln_b0_flist[pp].to_data_frame()
        elif f == 'rot_b0':
            df = rot_b0_flist[pp].to_data_frame()
        elif f == 'rot_b1':
            df = rot_b1_flist[pp].to_data_frame()
        elif f == 'mir_b0':
            df = mir_b0_flist[pp].to_data_frame()
        elif f == 'mir_b1':
            df = mir_b1_flist[pp].to_data_frame()
        elif f == 'rdm_b0':
            df = rdm_b0_flist[pp].to_data_frame()
        elif f == 'rdm_b1':
            df = rdm_b1_flist[pp].to_data_frame()
             
        time = df['time'].tolist()
        df = df[['FCz', 'Fz', 'Cz', 'Pz']]
        #df = df[['C3', 'C4']]
        ppdat = []
        for timept in range(0,800):
            subdf = df.loc[timept,:].mean()
            ppdat.append(subdf)
    
        newdf['pp%03d'% pp] = ppdat
    
    newdf['time'] = time
    newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_EarlyLate_%s_P3.csv' % (f))
    newdf.to_csv(newdf_filename)

In [ ]:
#plot FRN/ERN here using MNE functions
#channels = ['FCz', 'Fz']
channels = ['FCz', 'Fz', 'Cz', 'Pz']
#channels = ['C3']
#channels = ['C4']
# channels = ['Oz','O1','O2']
evokeds = dict(aln = aln_b0_flist, 
               rot_b0 = rot_b0_flist,
               rot_b1 = rot_b1_flist, 
               mir_b0 = mir_b0_flist, 
               mir_b1 = mir_b1_flist,
               rdm_b0 = rdm_b0_flist,
               rdm_b1 = rdm_b1_flist)
#plot
mne.viz.plot_compare_evokeds(evokeds, picks=channels, combine = 'mean', legend = 'lower right', ylim=dict(eeg=[-15, 5]), ci=True)